# Data Exploration
Fabian

In [1]:
import pandas as pd
import numpy as np
import sweetviz as sv
import os

path = r"./../dressipi_recsys2022"

In [2]:
# load DFs
df_purchases = pd.read_csv(os.path.join(path,"train_purchases.csv"))
df_sessions = pd.read_csv(os.path.join(path,"train_sessions.csv"))
df_item_features = pd.read_csv(os.path.join(path,"item_features.csv"))

In [3]:
# sweet viz
# analyze_report = sv.analyze(df_purchases)
# analyze_report.show_html('df_purchases.html', open_browser=True)

# analyze_report = sv.analyze(df_sessions)
# analyze_report.show_html('df_sessions.html', open_browser=True)

# analyze_report = sv.analyze(df_item_features)
# analyze_report.show_html('df_item_features.html', open_browser=True)

## Q1: How much is the overlap of purchases if the session is similar?
What I did:
- store train sessions in dictionary
- for every train session find "closest" train sample
- see how much the purchases overlap

Findings:
- overlap of sessions and purchases is very small
- item to item similarity will be neccesary to predict many items and not just do user-user similarity and see what similar user bought


In [4]:
## helper functions

def overlap_error(a, b):
    # this should be more sophisticated^^   
    err = 0
    for el in a:
        if el not in b:
            err += 1
        else:
            err-=1000
    
    for el in b:
        if el not in a:
            err += 1

    return err


def get_purchases_from_sid(sid):
    n_items = max(df_item_features["item_id"].unique())
    
    item_list = list()
    
    for row in df_purchases.iterrows():
        sid_p = row[1]["session_id"]
        
        if sid_p > sid:
            break
        
        iid = row[1]["item_id"]
        if sid_p == sid:
            item_list.append(iid)
    
    return item_list


In [5]:
# set parameters
cutoff = 1e5
n_items = max(df_item_features["item_id"].unique())

# store train sessions in dictionary to quickly look them up
ses_dic_train = dict()
for row in df_sessions.iterrows():
    sid = row[1]["session_id"]
    if sid > cutoff:
        break # use as train
        
    iid = row[1]["item_id"]
    if sid not in ses_dic_train.keys():
        ses_dic_train[sid] = list()
    ses_dic_train[sid].append(iid)
    

In [6]:
# ?Maybe?: prune train samples (those that overlap too hard ) 


In [7]:
# loadt test sessions
ses_dic_test = dict()
for row in df_sessions.iterrows():
    sid = row[1]["session_id"]
    if sid < cutoff:
        continue
    elif sid > cutoff+100: 
        break
        
    iid = row[1]["item_id"]
    if sid not in ses_dic_test.keys():
        ses_dic_test[sid] = list()
    ses_dic_test[sid].append(iid)
    

In [8]:
# find closest train sample for every test sample and print purchase overlap?
for i, sid_test in enumerate(ses_dic_test):
    print("sid", sid_test)
    s_test = ses_dic_test[sid_test]
    
    cur_error = np.inf
    cur_sid = -1
    
    full_list = list()
    # TODO: full_list_with_session_items?
    
    for sid_train in ses_dic_train:
        s_train = ses_dic_train[sid_train]
        error = overlap_error(s_test, s_train)
                
        if error < cur_error:
            cur_error = error
            cur_sid = sid_train
        
        if error < 0:
            full_list += get_purchases_from_sid(sid_train)
    
    test_purchases = get_purchases_from_sid(sid_test)
    pred_purchases = get_purchases_from_sid(cur_sid)
    
    print("Results:")
    
    if True or overlap_error(test_purchases, full_list) < 0:
        print("session")
        print(sid_test, s_test)
        print(cur_sid, ses_dic_train[cur_sid])
        print("error: ", overlap_error(s_test, ses_dic_train[cur_sid]))
        print("\npurchases")
        print(sid_test, test_purchases)
        print(cur_sid, pred_purchases)
        print("error: ", overlap_error(test_purchases, pred_purchases))

        print("\nfull list error (adding every purchase to a list where the session had overlap):")
        print("full_list", full_list)        
        print("error: ", overlap_error(test_purchases, full_list))
        
        print("######\n\n")
    
    if i >= 1:
        break
    
print("Done")

sid 100004
Results:
session
100004 [6942, 20268]
30434 [6942]
error:  -999

purchases
100004 [5418]
30434 [24454]
error:  2

full list error (adding every purchase to a list where the session had overlap):
full_list [2504, 17418, 23075, 11913, 15957, 8474, 14143, 19514, 23299, 23583, 17779, 5217, 740, 9478, 13815, 326, 24454, 9921, 18573, 580, 6145, 4573, 4005, 11132, 27193, 4540, 17166, 18058, 1623, 27455, 27931, 12479, 23705, 598, 3591, 6788, 24415, 27931, 5243, 1679, 26367]
error:  42
######


sid 100007
Results:
session
100007 [3911, 7756, 20688, 6274, 2465, 22017, 11132, 9184, 6274, 8907, 1018]
30977 [3985, 16484, 10478, 6274, 4005, 11132, 13106, 25446, 27903, 12746]
error:  -2984

purchases
100007 [18417]
30977 [22275]
error:  2

full list error (adding every purchase to a list where the session had overlap):
full_list [11065, 21781, 8060, 2504, 7035, 19044, 19160, 15373, 2100, 11619, 10111, 13424, 6261, 20843, 16427, 13296, 12511, 9427, 15687, 8861, 12781, 24593, 18657, 25250, 9